In [132]:
# !pip install tensorflow

In [133]:
# !pip install xgboost

In [134]:
# !pip install numpy==1.24.3

In [135]:
# Import Libraries and Dependencies
import pandas as pd
import numpy as np
import re
# import spacy
import subprocess
import os
import matplotlib.pyplot as plt
import seaborn as sns  
from sklearn.model_selection import train_test_split    # To split the dataset into train and test sets
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder   # For scaling and encoding categorical variables
from sklearn.compose import ColumnTransformer           # To apply transformers to specific columns
from sklearn.impute import SimpleImputer                # For handling missing data
from sklearn.ensemble import RandomForestClassifier     # Random Forest Classifier
from sklearn.linear_model import LogisticRegression     # Logistic Regression
from sklearn.svm import SVC                            # Support Vector Classifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score # For evaluating the model
import joblib
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dense, Dropout, Input, Concatenate, BatchNormalization

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np


In [136]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 


In [137]:
# Read in the CSV file
disease_df = pd.read_csv('Resources/disease_symptoms.csv')

In [138]:
# Rename Disease column to Prognosis
disease_df.rename(columns={'Disease': 'Prognosis'}, inplace=True)

In [139]:

# Expand Symptoms into separate columns
disease_df_expanded = disease_df["Symptoms"].str.split(", ", expand=True)

In [140]:

# Rename columns dynamically
disease_df_expanded.columns = [f"Symptom_{i+1}" for i in range(disease_df_expanded.shape[1])]

In [141]:

# Concatenate with Prognosis column
disease_df_final = pd.concat([disease_df["Prognosis"], disease_df_expanded], axis=1)

In [142]:

# Reshape from wide to long format
disease_df_long = disease_df_final.melt(id_vars="Prognosis", value_name="Symptom").drop(columns=["variable"])

In [143]:
# Drop NaN values before encoding**
disease_df_long = disease_df_long.dropna(subset=["Symptom"])

In [144]:
# **Fix: Trim whitespace from symptoms**
disease_df_long["Symptom"] = disease_df_long["Symptom"].str.strip()

In [145]:
# One-hot encode symptoms properly
disease_df_encoded = pd.get_dummies(disease_df_long, columns=["Symptom"])

In [146]:
# **Fix: Ensure index is reset**
disease_df_encoded = disease_df_encoded.reset_index(drop=True)

In [147]:
# sort alphabetically by Prognosis
disease_df_encoded = disease_df_encoded.sort_values("Prognosis")

In [148]:
# disease_df_encoded.set_index('Prognosis', inplace=True)


In [149]:
#reset the index 
disease_df_encoded = disease_df_encoded.reset_index(drop=True)
#Display the result
print(disease_df_encoded.shape)
disease_df_encoded.head(50)

(511, 263)


,Prognosis,Symptom_abdominal cramps,Symptom_abdominal pain,Symptom_abdominal swelling,Symptom_abnormal fatigue,Symptom_abnormal fluid accumulation in the abdomen,Symptom_absence of bowel movements for days,Symptom_absence of menstruation,Symptom_absent speech,Symptom_acid reflux,...,Symptom_waking up too early,Symptom_walking difficulty,Symptom_weakness,Symptom_weight gain,Symptom_weight loss,Symptom_wheezing,Symptom_white patches on the skin,Symptom_white pupil,Symptom_widespread itching,Symptom_yellowing of the skin and eyes
0,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,Alopecia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [150]:
# remove Symptom_ in the column names
disease_df_encoded.columns = disease_df_encoded.columns.str.replace('Symptom_', '')

# Display the result
disease_df_encoded.head(50)

,Prognosis,abdominal cramps,abdominal pain,abdominal swelling,abnormal fatigue,abnormal fluid accumulation in the abdomen,absence of bowel movements for days,absence of menstruation,absent speech,acid reflux,...,waking up too early,walking difficulty,weakness,weight gain,weight loss,wheezing,white patches on the skin,white pupil,widespread itching,yellowing of the skin and eyes
0,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Acetonemia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,Acute Bronchitis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,Alopecia,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [151]:
disease_df_encoded.shape

(511, 263)

In [152]:
# read in prognosis csv file
prognosis_df = pd.read_csv('Resources/symbipredict_2022.csv')

# Display the result
prognosis_df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal Infection


In [153]:
# Prognosis column to first column in the dataset
prognosis_df = prognosis_df[["prognosis"] + [col for col in prognosis_df.columns if col != "prognosis"]]
#sort the dataset alphabetically by prognosis
prognosis_df = prognosis_df.sort_values("prognosis")
#reset the index
# prognosis_df = prognosis_df.reset_index(drop=True)
prognosis_df.tail()

,prognosis,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
2537,Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3316,Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3890,Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2417,Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
364,Vertigo,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
# prognosis_df.set_index('prognosis', inplace=True)


In [155]:
prognosis_df.head()

,prognosis,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
2113,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
884,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
885,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2712,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
886,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
# rename prognosis column to Prognosis
prognosis_df.rename(columns={'prognosis': 'Prognosis'}, inplace=True)
# reset the index
prognosis_df = prognosis_df.reset_index(drop=True)

In [157]:
prognosis_df.head()

,Prognosis,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AIDS,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
prognosis_df.shape

(4961, 133)

In [159]:
prognosis_df.describe()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
count,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,...,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000,4961.000000
mean,0.137875,0.159847,0.021971,0.045152,0.021971,0.162266,0.139085,0.045152,0.045152,0.021971,...,0.021971,0.021971,0.021971,0.023181,0.023181,0.023181,0.023181,0.023181,0.023181,0.023181
std,0.344804,0.366501,0.146605,0.207659,0.146605,0.368732,0.346070,0.207659,0.207659,0.146605,...,0.146605,0.146605,0.146605,0.150493,0.150493,0.150493,0.150493,0.150493,0.150493,0.150493
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [160]:
prognosis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4961 entries, 0 to 4960
Columns: 133 entries, Prognosis to yellow_crust_ooze
dtypes: int64(132), object(1)
memory usage: 5.0+ MB


In [161]:
prognosis_df.dtypes

Prognosis               object
itching                  int64
skin_rash                int64
nodal_skin_eruptions     int64
continuous_sneezing      int64
                         ...  
small_dents_in_nails     int64
inflammatory_nails       int64
blister                  int64
red_sore_around_nose     int64
yellow_crust_ooze        int64
Length: 133, dtype: object

In [162]:
# read in prognosis csv file
final_train_df = pd.read_csv('Resources/FInal_Train_Data.csv')

# Display the result
final_train_df.head()

,Unnamed: 0,pain chest,shortness of breath,dizziness,asthenia,fall,syncope,vertigo,sweat sweating increased,palpitation,...,prodrome,hypoproteinemia,alcohol binge episode,abdomen acute,air fluid level,catching breath,large-for-dates fetus,immobile,homicidal thoughts,prognosis
0,0,0,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,hypertensive disease
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,diabetes
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"depression mental , depressive disorder"
3,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,"coronary arteriosclerosis ,coronary heart disease"
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pneumonia


In [163]:
final_train_df.shape

(2564, 402)

In [164]:
#drop 'Unnamed: 0' column from the final_train_df dataframe
final_train_df.drop('Unnamed: 0', axis=1, inplace=True)

In [165]:
# rename prognosis column to Prognosis
final_train_df.rename(columns={'prognosis': 'Prognosis'}, inplace=True)

# make prognosis column the first column in the dataset
final_train_df = final_train_df[["Prognosis"] + [col for col in final_train_df.columns if col != "Prognosis"]]

In [166]:
final_train_df.sort_values('Prognosis', inplace=True)

In [167]:
# reset the index
final_train_df = final_train_df.reset_index(drop=True)

In [168]:
#display the first 5 rows of the dataframe
final_train_df.head()

,Prognosis,pain chest,shortness of breath,dizziness,asthenia,fall,syncope,vertigo,sweat sweating increased,palpitation,...,feces in rectum,prodrome,hypoproteinemia,alcohol binge episode,abdomen acute,air fluid level,catching breath,large-for-dates fetus,immobile,homicidal thoughts
0,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alzheimer's disease,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [169]:
final_train_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2564 entries, 0 to 2563
Columns: 401 entries, Prognosis to homicidal thoughts
dtypes: int64(400), object(1)
memory usage: 7.8+ MB


In [170]:
final_train_df.describe()

,pain chest,shortness of breath,dizziness,asthenia,fall,syncope,vertigo,sweat sweating increased,palpitation,nausea,...,feces in rectum,prodrome,hypoproteinemia,alcohol binge episode,abdomen acute,air fluid level,catching breath,large-for-dates fetus,immobile,homicidal thoughts
count,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,...,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000,2564.000000
mean,0.064353,0.124415,0.022621,0.074103,0.026911,0.050312,0.015991,0.040952,0.035101,0.060062,...,0.003510,0.002730,0.001950,0.003120,0.002730,0.001560,0.001950,0.001560,0.001170,0.002340
std,0.245428,0.330119,0.148721,0.261989,0.161855,0.218631,0.125463,0.198217,0.184072,0.237649,...,0.059154,0.052189,0.044125,0.055782,0.052189,0.039475,0.044125,0.039475,0.034193,0.048327
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [171]:
final_train_df.dtypes

Prognosis                object
pain chest                int64
shortness of breath       int64
dizziness                 int64
asthenia                  int64
                          ...  
air fluid level           int64
catching breath           int64
large-for-dates fetus     int64
immobile                  int64
homicidal thoughts        int64
Length: 401, dtype: object

In [ ]:
# export the final_train_df, prognosis_df, and disease_df_encoded to a csv file
final_train_df.to_csv('Final_Resources/final_train_df.csv', index=False)
prognosis_df.to_csv('Final_Resources/prognosis_df.csv', index=False)
disease_df_encoded.to_csv('Final_Resources/disease_df_encoded.csv', index=False)

In [172]:
# Display Prognosis columns from each dataset
print("Prognosis from prognosis_df:")
print(prognosis_df['Prognosis'].unique())
print("\nCount:", len(prognosis_df['Prognosis'].unique()))

print("\nPrognosis from disease_df_encoded:")
print(disease_df_encoded['Prognosis'].unique())
print("\nCount:", len(disease_df_encoded['Prognosis'].unique()))

print("\nPrognosis from final_train_df:")
print(final_train_df['Prognosis'].unique())
print("\nCount:", len(final_train_df['Prognosis'].unique()))

# Compare the sets of prognoses
prognosis_set1 = set(prognosis_df['Prognosis'])
prognosis_set2 = set(disease_df_encoded['Prognosis'])
prognosis_set3 = set(final_train_df['Prognosis'])

print("\nOverlapping prognoses between all datasets:")
common_prognoses = prognosis_set1.intersection(prognosis_set2).intersection(prognosis_set3)
print(len(common_prognoses))

print("\nUnique to prognosis_df:")
print(prognosis_set1 - prognosis_set2.union(prognosis_set3))

print("\nUnique to disease_df_encoded:")
print(prognosis_set2 - prognosis_set1.union(prognosis_set3))

print("\nUnique to final_train_df:")
print(prognosis_set3 - prognosis_set1.union(prognosis_set2))

Prognosis from prognosis_df:
['AIDS' 'Acne' 'Alcoholic Hepatitis' 'Allergy' 'Arthritis'
 'Bronchial Asthma' 'Cervical Spondylosis' 'Chickenpox'
 'Chronic Cholestasis' 'Common Cold' 'Dengue' 'Diabetes '
 'Dimorphic Hemmorhoids (piles)' 'Drug Reaction' 'Fungal Infection' 'GERD'
 'Gastroenteritis' 'Heart Attack' 'Hepatitis A' 'Hepatitis B'
 'Hepatitis C' 'Hepatitis D' 'Hepatitis E' 'Hypertension '
 'Hyperthyroidism' 'Hypoglycemia' 'Hypothyroidism' 'Impetigo' 'Jaundice'
 'Malaria' 'Migraine' 'Osteoarthritis' 'Paralysis (brain hemorrhage)'
 'Peptic Ulcer Disease' 'Pneumonia' 'Psoriasis' 'Tuberculosis' 'Typhoid'
 'Urinary Tract Infection' 'Varicose Veins' 'Vertigo']

Count: 41

Prognosis from disease_df_encoded:
['Acetonemia' 'Acute Bronchitis' 'Alopecia' 'Amenorrhea' 'Amnesia'
 'Andropause' 'Anemia' 'Angina Pectoris' 'Angioma' 'Anorexia' 'Anthrax'
 'Anuria' 'Anxiety' 'Appendicitis' 'Asthma' 'Astigmatism' 'Autism'
 'Bloating' 'Burn' 'Candidiasis' 'Cataract' 'Cavity' 'Cellulite'
 'Cervical Ca

In [173]:
print("Number of unique conditions in each dataset:")
print("prognosis_df:", len(prognosis_df['Prognosis'].unique()))
print("disease_df_encoded:", len(disease_df_encoded['Prognosis'].unique()))
print("final_train_df:", len(final_train_df['Prognosis'].unique()))

Number of unique conditions in each dataset:
prognosis_df: 41
disease_df_encoded: 108
final_train_df: 133


In [174]:
# Create a more comprehensive mapping dictionary
condition_mapping = {
    # From prognosis_df to final_train_df format
    'AIDS': 'acquired immuno-deficiency syndrome HIV hiv infections',
    'Arthritis': 'arthritis',
    'Bronchial Asthma': 'asthma',
    'Diabetes ': 'diabetes',
    'Gastroenteritis': 'gastroenteritis',
    'Heart Attack': 'myocardial infarction',
    'Hepatitis B': 'hepatitis B',
    'Hepatitis C': 'hepatitis C',
    'Hepatitis D': 'hepatitis',
    'Hepatitis E': 'hepatitis',
    'Hypertension ': 'hypertensive disease',
    'Hypothyroidism': 'hypothyroidism',
    'Hyperthyroidism': 'hyperthyroidism',
    'Migraine': 'migraine disorders',
    'Osteoarthritis': 'degenerative polyarthritis',
    'Paralysis (brain hemorrhage)': 'accident cerebrovascular',
    'Peptic Ulcer Disease': 'ulcer peptic',
    'Pneumonia': 'pneumonia',
    'Tuberculosis': 'infection',
    'Urinary Tract Infection': 'infection urinary tract',
    'Varicose Veins': 'peripheral vascular disease',
    'Vertigo': 'delirium',
    
    # From disease_df_encoded to final_train_df format
    'Acute Bronchitis': 'bronchitis',
    'Amnesia': 'confusion',
    'Anemia': 'anemia',
    'Anxiety': 'anxiety state',
    'Asthma': 'asthma',
    'Autism': 'affect labile',
    'Chronic Bronchitis': 'bronchitis',
    'Cirrhosis': 'cirrhosis',
    'Colitis': 'colitis',
    'Gastroenteritis': 'gastroenteritis',
    'Glaucoma': 'glaucoma',
    'Gout': 'gout',
    'Hemorrhoids': 'hemorrhoids',
    'Hepatitis': 'hepatitis',
    'Hernia': 'hernia',
    'Hypertension': 'hypertensive disease',
    'Hypothyroidism': 'hypothyroidism',
    'Osteoporosis': 'osteoporosis',
    'Pancreatitis': 'pancreatitis',
    'Parkinson': 'parkinson disease',
    'Pneumothorax': 'pneumothorax',
    'Sinusitis': 'upper respiratory infection',
    'Stroke': 'accident cerebrovascular',
    'Candidiasis': 'candidiasis oralcandidiasis',
    'Cervical Cancer': 'malignant neoplasms',
    'Liver Cancer': 'primary carcinoma of the liver cells',
    'Lung Cancer': 'malignant neoplasm of lung carcinoma of lung',
    'Prostate Cancer': 'malignant neoplasm of prostate carcinoma prostate',
    'Skin Cancer': 'malignant neoplasms'
}



In [175]:
# Apply mapping to standardize conditions
prognosis_df['Prognosis_Mapped'] = prognosis_df['Prognosis'].map(condition_mapping).fillna(prognosis_df['Prognosis'])
disease_df_encoded['Prognosis_Mapped'] = disease_df_encoded['Prognosis'].map(condition_mapping).fillna(disease_df_encoded['Prognosis'])
final_train_df['Prognosis_Mapped'] = final_train_df['Prognosis']

In [176]:
# Check the overlap after mapping
print("\nAfter mapping, common conditions between datasets:")
common_conditions = set(prognosis_df['Prognosis_Mapped']) & set(disease_df_encoded['Prognosis_Mapped']) & set(final_train_df['Prognosis_Mapped'])
print(f"Number of common conditions: {len(common_conditions)}")
print("\nCommon conditions:")
for condition in sorted(common_conditions):
    print(condition)


After mapping, common conditions between datasets:
Number of common conditions: 10

Common conditions:
asthma
delirium
gastroenteritis
hepatitis
hypertensive disease
hypothyroidism
infection urinary tract
migraine disorders
peripheral vascular disease
pneumonia


In [177]:
# Filter datasets to keep only common conditions
prognosis_df_filtered = prognosis_df[prognosis_df['Prognosis_Mapped'].isin(common_conditions)]
disease_df_encoded_filtered = disease_df_encoded[disease_df_encoded['Prognosis_Mapped'].isin(common_conditions)]
final_train_df_filtered = final_train_df[final_train_df['Prognosis_Mapped'].isin(common_conditions)]

In [178]:
# Show the final sizes
print("\nFinal dataset sizes after filtering:")
print("prognosis_df:", len(prognosis_df_filtered))
print("disease_df_encoded:", len(disease_df_encoded_filtered))
print("final_train_df:", len(final_train_df_filtered))


Final dataset sizes after filtering:
prognosis_df: 1331
disease_df_encoded: 56
final_train_df: 178


In [179]:
# reset the index for the filtered datasets
prognosis_df_filtered = prognosis_df_filtered.reset_index(drop=True)
disease_df_encoded_filtered = disease_df_encoded_filtered.reset_index(drop=True)
final_train_df_filtered = final_train_df_filtered.reset_index(drop=True)

In [180]:
prognosis_df_filtered.head(100)

,Prognosis,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,Prognosis_Mapped
0,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
1,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
2,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
3,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
4,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
96,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
97,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
98,Bronchial Asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma


In [181]:
disease_df_encoded_filtered.head(10)

,Prognosis,abdominal cramps,abdominal pain,abdominal swelling,abnormal fatigue,abnormal fluid accumulation in the abdomen,absence of bowel movements for days,absence of menstruation,absent speech,acid reflux,...,walking difficulty,weakness,weight gain,weight loss,wheezing,white patches on the skin,white pupil,widespread itching,yellowing of the skin and eyes,Prognosis_Mapped
0,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,asthma
1,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,asthma
2,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,asthma
3,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,asthma
4,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,asthma
5,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,asthma
6,Asthma,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,asthma
7,Gastroenteritis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,gastroenteritis
8,Gastroenteritis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,gastroenteritis
9,Gastroenteritis,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,gastroenteritis


In [182]:
final_train_df_filtered.head(10)

,Prognosis,pain chest,shortness of breath,dizziness,asthenia,fall,syncope,vertigo,sweat sweating increased,palpitation,...,prodrome,hypoproteinemia,alcohol binge episode,abdomen acute,air fluid level,catching breath,large-for-dates fetus,immobile,homicidal thoughts,Prognosis_Mapped
0,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
1,asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
2,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
3,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
4,asthma,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
5,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
6,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
7,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
8,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma
9,asthma,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asthma


In [183]:
prognosis_df_filtered.value_counts()

Prognosis                itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  muscle_wasting  vomiting  burning_micturition  spotting_ urination  fatigue  weight_gain  anxiety  cold_hands_and_feets  mood_swings  weight_loss  restlessness  lethargy  patches_in_throat  irregular_sugar_level  cough  high_fever  sunken_eyes  breathlessness  sweating  dehydration  indigestion  headache  yellowish_skin  dark_urine  nausea  loss_of_appetite  pain_behind_the_eyes  back_pain  constipation  abdominal_pain  diarrhoea  mild_fever  yellow_urine  yellowing_of_eyes  acute_liver_failure  fluid_overload  swelling_of_stomach  swelled_lymph_nodes  malaise  blurred_and_distorted_vision  phlegm  throat_irritation  redness_of_eyes  sinus_pressure  runny_nose  congestion  chest_pain  weakness_in_limbs  fast_heart_rate  pain_during_bowel_movements  pain_in_anal_region  bloody_stool  irritation_in_anus  neck_pain  dizziness  cram

In [184]:
prognosis_df_filtered['Prognosis_Mapped'].value_counts()

Prognosis_Mapped
hepatitis                      242
asthma                         121
gastroenteritis                121
hypertensive disease           121
hypothyroidism                 121
migraine disorders             121
pneumonia                      121
infection urinary tract        121
peripheral vascular disease    121
delirium                       121
Name: count, dtype: int64

In [185]:
# Separate features and target labels
y = prognosis_df_filtered["Prognosis_Mapped"].to_numpy()

X1 = prognosis_df_filtered.drop(columns=["Prognosis", "Prognosis_Mapped"]).to_numpy()
X2 = disease_df_encoded_filtered.drop(columns=["Prognosis", "Prognosis_Mapped"]).to_numpy()
X3 = final_train_df_filtered.drop(columns=["Prognosis", "Prognosis_Mapped"]).to_numpy()

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# First check the data
print("Original shapes before any processing:")
print("X1:", X1.shape)
print("X2:", X2.shape)
print("X3:", X3.shape)
print("y:", y.shape)

Original shapes before any processing:
X1: (1331, 132)
X2: (56, 262)
X3: (178, 400)
y: (1331,)


In [186]:
print(X1[0:10])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [187]:
# 2. Get the size of largest dataset
max_samples = max(len(X1), len(X2), len(X3))
print("\nTarget samples:", max_samples)

# Upsample smaller datasets
if len(X1) < max_samples:
    indices = np.random.choice(len(X1), max_samples - len(X1))
    X1_extra = X1[indices]
    y_extra = y[indices]
    X1 = np.vstack([X1, X1_extra])
    y = np.concatenate([y, y_extra])

if len(X2) < max_samples:
    indices = np.random.choice(len(X2), max_samples - len(X2))
    X2_extra = X2[indices]
    X2 = np.vstack([X2, X2_extra])

if len(X3) < max_samples:
    indices = np.random.choice(len(X3), max_samples - len(X3))
    X3_extra = X3[indices]
    X3 = np.vstack([X3, X3_extra])

print("\nFinal shapes after upsampling:")
print("X1:", X1.shape)
print("X2:", X2.shape)
print("X3:", X3.shape)
print("y_encoded:", y_encoded.shape)

# Encode labels after upsampling
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


Target samples: 1331

Final shapes after upsampling:
X1: (1331, 132)
X2: (1331, 262)
X3: (1331, 400)
y_encoded: (1331,)


In [188]:
# 2. Split and scale with stratification
X1_train, X1_test, y_train, y_test = train_test_split(X1, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
X2_train, X2_test = train_test_split(X2, test_size=0.2, random_state=42)
X3_train, X3_test = train_test_split(X3, test_size=0.2, random_state=42)

# 3. Scale the data
scaler1 = StandardScaler()
scaler2 = StandardScaler()
scaler3 = StandardScaler()

X1_train_scaled = scaler1.fit_transform(X1_train)
X1_test_scaled = scaler1.transform(X1_test)

X2_train_scaled = scaler2.fit_transform(X2_train)
X2_test_scaled = scaler2.transform(X2_test)

X3_train_scaled = scaler3.fit_transform(X3_train)
X3_test_scaled = scaler3.transform(X3_test)

In [189]:
# 4. Create datasets with small batch size
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "prognosis_input": X1_train_scaled,
        "disease_input": X2_train_scaled,
        "final_train_input": X3_train_scaled
    },
    y_train
)).shuffle(1000).batch(64)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "prognosis_input": X1_test_scaled,
        "disease_input": X2_test_scaled,
        "final_train_input": X3_test_scaled
    },
    y_test
)).batch(64)

In [199]:
# print train and test dataset shapes
print("Train dataset shape:", X1_train_scaled.shape, X2_train_scaled.shape, X3_train_scaled.shape)
train_dataset


Train dataset shape: (1064, 132) (1064, 262) (1064, 400)


<_BatchDataset element_spec=({'prognosis_input': TensorSpec(shape=(None, 132), dtype=tf.float64, name=None), 'disease_input': TensorSpec(shape=(None, 262), dtype=tf.float64, name=None), 'final_train_input': TensorSpec(shape=(None, 400), dtype=tf.float64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [190]:
# # 4. Build simpler model
# input1 = Input(shape=(132,), name="prognosis_input")
# input2 = Input(shape=(262,), name="disease_input")
# input3 = Input(shape=(400,), name="final_train_input")

# # Process each input
# x1 = Dense(64, activation="relu")(input1)
# x1 = BatchNormalization()(x1)
# x1 = Dropout(0.2)(x1)

# x2 = Dense(64, activation="relu")(input2)
# x2 = BatchNormalization()(x2)
# x2 = Dropout(0.2)(x2)

# x3 = Dense(64, activation="relu")(input3)
# x3 = BatchNormalization()(x3)
# x3 = Dropout(0.2)(x3)

# # Merge and process
# merged = Concatenate()([x1, x2, x3])
# x = Dense(128, activation="relu")(merged)
# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)

# output = Dense(len(np.unique(y_encoded)), activation="softmax")(x)

In [ ]:
# 4. Build simpler model
input1 = Input(shape=(132,), name="prognosis_input")
input2 = Input(shape=(262,), name="disease_input")
input3 = Input(shape=(400,), name="final_train_input")

# Process each input
x1 = Dense(64, activation="relu")(input1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.2)(x1)

x2 = Dense(64, activation="relu")(input2)
x2 = BatchNormalization()(x2)
x2 = Dropout(0.2)(x2)

x3 = Dense(64, activation="relu")(input3)
x3 = BatchNormalization()(x3)
x3 = Dropout(0.2)(x3)

# Merge and process
merged = Concatenate()([x1, x2, x3])
x = Dense(128, activation="relu")(merged)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)

output = Dense(len(np.unique(y_encoded)), activation="softmax")(x)

In [200]:
# 5. Compile with different optimizer settings
model = Model(inputs=train_dataset, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# 6. Train with different callback settings
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=5,
    min_lr=0.00001,
    verbose=1
)

# 7. Train
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=test_dataset,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

ValueError: All `inputs` values must be KerasTensors. Received: inputs=<_BatchDataset element_spec=({'prognosis_input': TensorSpec(shape=(None, 132), dtype=tf.float64, name=None), 'disease_input': TensorSpec(shape=(None, 262), dtype=tf.float64, name=None), 'final_train_input': TensorSpec(shape=(None, 400), dtype=tf.float64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))> including invalid value <_BatchDataset element_spec=({'prognosis_input': TensorSpec(shape=(None, 132), dtype=tf.float64, name=None), 'disease_input': TensorSpec(shape=(None, 262), dtype=tf.float64, name=None), 'final_train_input': TensorSpec(shape=(None, 400), dtype=tf.float64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))> of type <class 'tensorflow.python.data.ops.batch_op._BatchDataset'>

In [192]:
# print final accuracy
print(f"Final accuracy: {history.history['accuracy'][-1]}")
print(f"Final validation accuracy: {history.history['val_accuracy'][-1]}")

# 8. Evaluate
y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))



Final accuracy: 0.9990601539611816
Final validation accuracy: 1.0


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
                             precision    recall  f1-score   support

                     asthma       1.00      1.00      1.00        24
                   delirium       1.00      1.00      1.00        24
            gastroenteritis       1.00      1.00      1.00        24
                  hepatitis       1.00      1.00      1.00        49
       hypertensive disease       1.00      1.00      1.00        25
             hypothyroidism       1.00      1.00      1.00        24
    infection urinary tract       1.00      1.00      1.00        24
         migraine disorders       1.00      1.00      1.00        25
peripheral vascular disease       1.00      1.00      1.00        24
                  pneumonia       1.00      1.00      1.00        24

                   accuracy                           1.00       267
                  macro avg       1.00      1.00      1.00       267
               weighted avg       1.00      1.00      1.00     

In [193]:
# 10. Print final metrics
test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")

y_pred = np.argmax(model.predict(test_dataset), axis=1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 11. Print model summary
print("\nModel Summary:")
model.summary()


Final Test Accuracy: 1.0000
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step



Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        24
           1       1.00      1.00      1.00        24
           2       1.00      1.00      1.00        24
           3       1.00      1.00      1.00        49
           4       1.00      1.00      1.00        25
           5       1.00      1.00      1.00        24
           6       1.00      1.00      1.00        24
           7       1.00      1.00      1.00        25
           8       1.00      1.00      1.00        24
           9       1.00      1.00      1.00        24

    accuracy                           1.00       267
   macro avg       1.00      1.00      1.00       267
weighted avg       1.00      1.00      1.00       267


Model Summary:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ prognosis_input     │ (None, 132)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ disease_input       │ (None, 262)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ final_train_input   │ (None, 400)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 64)        │      8,512 │ prognosis_input[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 64)        │     16,832 │ disease_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │     25,664 │ final_train_inpu… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_10[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_11[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64)        │        256 │ dense_12[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 64)        │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 64)        │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 192)       │          0 │ dropout_8[0][0],  │
│ (Concatenate)       │                   │            │ dropout_9[0][0],  │
│                     │                   │            │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128)       │     24,704 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_13[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 128)       │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 10)        │      1,290 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 233,568 (912.38 KB)

 Trainable params: 77,642 (303.29 KB)

 Non-trainable params: 640 (2.50 KB)

 Optimizer params: 155,286 (606.59 KB)